In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data/flights_test.csv')
df1 = pd.read_csv('data/flights_test.csv')

In [ ]:
df.columns

Index(['date_format', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance'],
      dtype='object')

In [ ]:
df1.drop(columns=['fl_date','branded_code_share','mkt_carrier',
'op_carrier_fl_num','dest_airport_id','mkt_carrier_fl_num','op_unique_carrier',
'tail_num','origin_airport_id','origin','dest','dup','flights'], inplace=True)

In [ ]:
df.rename(columns={'date_format': 'fl_date'}, inplace=True)

In [ ]:
df.head(5)

,fl_date,mkt_unique_carrier,origin_city_name,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-01,WN,"Ontario, CA","San Francisco, CA",1810,1945,95,363
1,2020-01-01,WN,"Ontario, CA","San Francisco, CA",1150,1320,90,363
2,2020-01-01,WN,"Ontario, CA","San Jose, CA",2020,2130,70,333
3,2020-01-01,WN,"Ontario, CA","San Jose, CA",1340,1455,75,333
4,2020-01-01,WN,"Ontario, CA","San Jose, CA",915,1035,80,333


In [ ]:
df.isnull().sum()

fl_date               0
mkt_unique_carrier    0
origin_city_name      0
dest_city_name        0
crs_dep_time          0
crs_arr_time          0
crs_elapsed_time      0
distance              0
dtype: int64

In [ ]:
#import TimeFormatted from function_helpers in folder data(change folder accordingly)
from data.function_helpers import TimeFormatted

In [ ]:
"""
extract hour from int format in 'crs_dep_time','crs_arr_time' columns
"""

df['crs_arr_time'] = df['crs_arr_time'].apply(TimeFormatted)
df['crs_dep_time'] = df['crs_dep_time'].apply(TimeFormatted)
df['crs_arr_time'] = pd.to_datetime(df['crs_arr_time'], format='%H:%M:%S').dt.hour
df['crs_dep_time'] = pd.to_datetime(df['crs_dep_time'], format='%H:%M:%S').dt.hour


In [ ]:
df.isnull().sum()

fl_date               0
mkt_unique_carrier    0
origin_city_name      0
dest_city_name        0
crs_dep_time          0
crs_arr_time          6
crs_elapsed_time      0
distance              0
dtype: int64

In [ ]:
df['crs_arr_time'] = df['crs_arr_time'].fillna(df['crs_arr_time'].mean())
df['crs_arr_time'] = df['crs_arr_time'].astype(int)

In [ ]:
df.isnull().sum()

fl_date               0
mkt_unique_carrier    0
origin_city_name      0
dest_city_name        0
crs_dep_time          0
crs_arr_time          0
crs_elapsed_time      0
distance              0
dtype: int64

In [ ]:
df.head(5)

,fl_date,mkt_unique_carrier,origin_city_name,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-01,WN,"Ontario, CA","San Francisco, CA",18,19,95,363
1,2020-01-01,WN,"Ontario, CA","San Francisco, CA",11,13,90,363
2,2020-01-01,WN,"Ontario, CA","San Jose, CA",20,21,70,333
3,2020-01-01,WN,"Ontario, CA","San Jose, CA",13,14,75,333
4,2020-01-01,WN,"Ontario, CA","San Jose, CA",9,10,80,333


## Origin flight test merge

In [ ]:
weather_df = pd.read_csv('flights_test_weather-2020.csv')

In [ ]:
weather_df.rename(columns={'city': 'origin_city_name', 'date': 'fl_date'}, inplace=True)
#weather_df.head(5)
weather_df['origin_city_condn'] = np.nan

In [ ]:
condition_dict = {'cloundy': ['Cloudy','Overcast'],
                  'sunny': ['Partly cloudy','Sunny'],
                  'low_snow' : ['Freezing drizzle','Patchy snow possible', 'Patchy light snow','Light snow'],
                  'snow': ['Light snow showers','Moderate or heavy snow with thunder','Light showers of ice pellets','Light sleet','Moderate snow','Patchy heavy snow','Patchy moderate snow','Moderate or heavy snow showers'],
                  'low_rainny': ['Mist','Patchy light drizzle','Light drizzle','Patchy rain possible','Light rain','Patchy sleet possible','Patchy light rain'],
                  'rainny': ['Light rain shower','Patchy freezing drizzle possible','Patchy light rain with thunder','Heavy rain at times','Light sleet showers','Moderate or heavy rain with thunder','Heavy rain','Light freezing rain','Moderate rain','Moderate or heavy rain shower','Moderate rain at times'],
                  'harzard': ['Thundery outbreaks possible','Blowing snow','Torrential rain shower','Freezing fog','Moderate or heavy freezing rain','Ice pellets','Fog','Moderate or heavy sleet','Heavy snow', 'Blizzard']
                 }

In [ ]:
for key, conditions in condition_dict.items():
    for condition in conditions:
        weather_df.loc[weather_df['condition'] == condition, "origin_city_condn"] = key

In [ ]:
weather_df.head(5)

,fl_date,origin_city_name,condition,origin_city_condn
0,2020-01-01,"Aberdeen, SD",Overcast,cloundy
1,2020-01-01,"Abilene, TX",Partly cloudy,sunny
2,2020-01-01,"Adak Island, AK",Moderate or heavy snow showers,snow
3,2020-01-01,"Aguadilla, PR",Patchy rain possible,low_rainny
4,2020-01-01,"Akron, OH",Heavy snow,harzard


In [ ]:
origin_df = df[['fl_date', 'origin_city_name']]
df.shape, weather_df.shape

((150623, 8), (4897, 4))

In [ ]:
origin_df.head(5)

,fl_date,origin_city_name
0,2020-01-01,"Ontario, CA"
1,2020-01-01,"Ontario, CA"
2,2020-01-01,"Ontario, CA"
3,2020-01-01,"Ontario, CA"
4,2020-01-01,"Ontario, CA"


In [ ]:
origin_weather_merged = pd.merge(origin_df, weather_df.drop_duplicates(),
                                 how="left", on=["fl_date","origin_city_name"])
origin_weather_merged['origin_city_condn'] = origin_weather_merged['origin_city_condn'].fillna(
                                                origin_weather_merged['origin_city_condn'].median)

In [ ]:
origin_weather_merged.shape, origin_weather_merged.isnull().sum()

((150623, 4),
 fl_date              0
 origin_city_name     0
 condition            0
 origin_city_condn    0
 dtype: int64)

In [ ]:
origin_weather_merged.head(5)

,fl_date,origin_city_name,condition,origin_city_condn
0,2020-01-01,"Ontario, CA",Partly cloudy,sunny
1,2020-01-01,"Ontario, CA",Partly cloudy,sunny
2,2020-01-01,"Ontario, CA",Partly cloudy,sunny
3,2020-01-01,"Ontario, CA",Partly cloudy,sunny
4,2020-01-01,"Ontario, CA",Partly cloudy,sunny


In [ ]:
origin_weather_merged.drop(columns='condition', inplace=True)

In [ ]:
origin_weather_merged.to_csv("origin_weather_flights_test.csv")

## Dest weather merge

In [ ]:
dest_df = df[['fl_date', 'dest_city_name']]
dest_df.shape,dest_df.head(5)

((150623, 2),
       fl_date     dest_city_name
 0  2020-01-01  San Francisco, CA
 1  2020-01-01  San Francisco, CA
 2  2020-01-01       San Jose, CA
 3  2020-01-01       San Jose, CA
 4  2020-01-01       San Jose, CA)

In [ ]:
#use the weather_df above just change origin to dest(extra caution when use weather_df again)
weather_df.rename(columns={'origin_city_name': 'dest_city_name', 'origin_city_condn': 'dest_city_cond'},inplace=True)
weather_df.shape, weather_df.head(5)

((4897, 4),
       fl_date   dest_city_name                       condition dest_city_cond
 0  2020-01-01     Aberdeen, SD                        Overcast        cloundy
 1  2020-01-01      Abilene, TX                   Partly cloudy          sunny
 2  2020-01-01  Adak Island, AK  Moderate or heavy snow showers           snow
 3  2020-01-01    Aguadilla, PR            Patchy rain possible     low_rainny
 4  2020-01-01        Akron, OH                      Heavy snow        harzard)

In [ ]:
#merge to get weather for dest city
dest_weather_merged = pd.merge(dest_df, weather_df.drop_duplicates(),
                               how="left", on=["fl_date","dest_city_name"])
dest_weather_merged['dest_city_cond'] = dest_weather_merged['dest_city_cond'].fillna(
                                                dest_weather_merged['dest_city_cond'].median)

In [ ]:
dest_weather_merged.shape, dest_weather_merged.isnull().sum()

((150623, 4),
 fl_date           0
 dest_city_name    0
 condition         0
 dest_city_cond    0
 dtype: int64)

In [ ]:
dest_weather_merged.drop(columns='condition', inplace=True)

In [ ]:
dest_weather_merged.head(5)

,fl_date,dest_city_name,dest_city_cond
0,2020-01-01,"San Francisco, CA",sunny
1,2020-01-01,"San Francisco, CA",sunny
2,2020-01-01,"San Jose, CA",cloundy
3,2020-01-01,"San Jose, CA",cloundy
4,2020-01-01,"San Jose, CA",cloundy


In [ ]:
dest_weather_merged.to_csv("dest_weather_flights_test.csv")

In [ ]:
origin_wth = pd.read_csv('origin_weather_flights_test.csv')
dest_wth = pd.read_csv('dest_weather_flights_test.csv')
origin_wth.drop(columns=['Unnamed: 0','condition'], inplace=True)
dest_wth.drop(columns='Unnamed: 0', inplace=True)
origin_wth.shape, origin_wth.head(5)

((150623, 3),
       fl_date origin_city_name origin_city_condn
 0  2020-01-01      Ontario, CA             sunny
 1  2020-01-01      Ontario, CA             sunny
 2  2020-01-01      Ontario, CA             sunny
 3  2020-01-01      Ontario, CA             sunny
 4  2020-01-01      Ontario, CA             sunny)

In [ ]:
dest_wth.shape, dest_wth.head(5)

((150623, 3),
       fl_date     dest_city_name dest_city_cond
 0  2020-01-01  San Francisco, CA          sunny
 1  2020-01-01  San Francisco, CA          sunny
 2  2020-01-01       San Jose, CA        cloundy
 3  2020-01-01       San Jose, CA        cloundy
 4  2020-01-01       San Jose, CA        cloundy)

In [ ]:
routes = df[['fl_date','origin_city_name', 'dest_city_name']]
routes.shape, routes.head(5)

((150623, 3),
       fl_date origin_city_name     dest_city_name
 0  2020-01-01      Ontario, CA  San Francisco, CA
 1  2020-01-01      Ontario, CA  San Francisco, CA
 2  2020-01-01      Ontario, CA       San Jose, CA
 3  2020-01-01      Ontario, CA       San Jose, CA
 4  2020-01-01      Ontario, CA       San Jose, CA)

In [ ]:
first_merged = pd.merge(routes, origin_wth.drop_duplicates(), how="left", on=["fl_date", "origin_city_name"])

In [ ]:
first_merged.isnull().sum()

fl_date              0
origin_city_name     0
dest_city_name       0
origin_city_condn    0
dtype: int64

In [ ]:
origin_dest_weather_fl_test = pd.merge(first_merged, dest_wth.drop_duplicates(), how="left", on=["fl_date", "dest_city_name"])

In [ ]:
origin_dest_weather_fl_test.shape

(150623, 5)

In [ ]:
origin_dest_weather_fl_test.head(5)

,fl_date,origin_city_name,dest_city_name,origin_city_condn,dest_city_cond
0,2020-01-01,"Ontario, CA","San Francisco, CA",sunny,sunny
1,2020-01-01,"Ontario, CA","San Francisco, CA",sunny,sunny
2,2020-01-01,"Ontario, CA","San Jose, CA",sunny,cloundy
3,2020-01-01,"Ontario, CA","San Jose, CA",sunny,cloundy
4,2020-01-01,"Ontario, CA","San Jose, CA",sunny,cloundy


In [ ]:
origin_dest_weather_fl_test.to_csv("origin_dest_weather_fl_test.csv")

In [ ]:
weather_fl = pd.read_csv('origin_dest_weather_fl_test.csv')

In [ ]:
weather_fl.drop(columns='Unnamed: 0',inplace=True)
weather_fl.head(5)

,fl_date,origin_city_name,dest_city_name,origin_city_condn,dest_city_cond
0,2020-01-01,"Ontario, CA","San Francisco, CA",sunny,sunny
1,2020-01-01,"Ontario, CA","San Francisco, CA",sunny,sunny
2,2020-01-01,"Ontario, CA","San Jose, CA",sunny,cloundy
3,2020-01-01,"Ontario, CA","San Jose, CA",sunny,cloundy
4,2020-01-01,"Ontario, CA","San Jose, CA",sunny,cloundy


In [ ]:
#Merging API weather Data to the Flighs_Traffic tables 
flights_test_2020 = pd.merge(df, weather_fl.drop_duplicates(), how ='left',
         on = ['fl_date','origin_city_name','dest_city_name'])

In [ ]:
flights_test_2020.shape

(150623, 10)

In [ ]:
flights_test_2020.to_csv('flights_test_2020.csv')

In [ ]:
flights_test_2020.columns

Index(['fl_date', 'mkt_unique_carrier', 'origin_city_name', 'dest_city_name',
       'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance',
       'origin_city_condn', 'dest_city_cond'],
      dtype='object')

In [ ]:
flights_test_2020.shape, flights_test_2020.columns

((150623, 11),
 Index(['mkt_unique_carrier', 'origin_city_name', 'dest_city_name',
        'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance',
        'origin_city_condn', 'dest_city_cond', 'day', 'day_Week'],
       dtype='object'))